Build Neural Network

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)  #move neural network to device
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1,28,28,device=device)   #random input
logits = model(X)   #executes model.forward
print(f"Raw predicted values for each class: \n{logits}\n")
pred_probab = nn.Softmax(dim=1)(logits)
print(f"After softmax: \n{pred_probab}\n")
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Raw predicted values for each class: 
tensor([[-0.0036, -0.0067,  0.0759,  0.0762,  0.0630, -0.0329,  0.0357,  0.0758,
          0.0508, -0.0323]], device='cuda:0', grad_fn=<AddmmBackward0>)

After softmax: 
tensor([[0.0966, 0.0963, 0.1046, 0.1046, 0.1032, 0.0938, 0.1005, 0.1046, 0.1020,
         0.0939]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

Predicted class: tensor([3], device='cuda:0')


nn.Flatten : convert 28*28 image to 784 pixel values

(Using minibatch of 3 images)

In [15]:
input_image = torch.rand(3,28,28)
print(f"Before flatten: {input_image.size()}\n")

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(f"After flatten: {flat_image.size()}")

Before flatten: torch.Size([3, 28, 28])

After flatten: torch.Size([3, 784])


nn.Linear

In [16]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU (Non-linear)

In [17]:
print(f"Before ReLU: \n{hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: \n{hidden1}")

Before ReLU: 
tensor([[ 0.2590, -0.3184, -0.0322, -0.6793, -0.2706, -0.1341, -0.2920,  0.4137,
         -0.4050, -0.0821,  0.2271,  0.3237, -0.6645,  0.2777, -0.5085,  0.0731,
          0.5503, -0.4521, -0.2857, -0.2812],
        [ 0.3351,  0.0044, -0.0483, -0.3780, -0.2539, -0.2003, -0.0210,  0.3894,
         -0.5235, -0.0826,  0.5889,  0.4377, -0.6859,  0.0795, -0.2018,  0.1436,
          0.5974, -0.5946, -0.1074, -0.6460],
        [ 0.4948, -0.0689, -0.3097, -0.0900, -0.3936, -0.0325,  0.2796,  0.4733,
         -0.3402,  0.3389,  0.5762,  0.3602, -0.5914,  0.0902, -0.0584,  0.1232,
          0.5258, -0.7994,  0.0625, -0.3169]], grad_fn=<AddmmBackward0>)


After ReLU: 
tensor([[0.2590, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4137, 0.0000,
         0.0000, 0.2271, 0.3237, 0.0000, 0.2777, 0.0000, 0.0731, 0.5503, 0.0000,
         0.0000, 0.0000],
        [0.3351, 0.0044, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3894, 0.0000,
         0.0000, 0.5889, 0.4377, 0.0000, 0.0795, 0.

nn.Softmax : values [-infty,infty] are scaled to [0,1], which are the predicted probabilities for each class.

In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)   #raw values [-infty, infty]

softmax = nn.Softmax(dim=1) #values along dimension1 must sum to 1
pred_probab = softmax(logits)
print(f"{pred_probab}")

tensor([[0.1011, 0.0835, 0.1170, 0.1349, 0.0840, 0.0874, 0.0828, 0.0796, 0.1011,
         0.1287],
        [0.1080, 0.0815, 0.1091, 0.1352, 0.0795, 0.0846, 0.0809, 0.0778, 0.1071,
         0.1363],
        [0.1073, 0.0995, 0.1005, 0.1217, 0.0811, 0.0938, 0.0884, 0.0890, 0.0929,
         0.1257]], grad_fn=<SoftmaxBackward0>)


Access model parameters

In [22]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} \n|-Size: {param.size()} \n|-Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight 
|-Size: torch.Size([512, 784]) 
|-Values: tensor([[-0.0102,  0.0032, -0.0306,  ...,  0.0256, -0.0049,  0.0313],
        [ 0.0045,  0.0154,  0.0156,  ..., -0.0203, -0.0067,  0.0265]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias 
|-Size: torch.Size([512]) 
|-Values: tensor([-0.0320, -0.0102], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight 
|-Size: torch.Size([512, 512]) 
|-Values: tensor([[-0.0005, -0.0153, -0.0317,  ..., -0.0086,  0.0005,  0.0265],
        [-0.0272,  0.0019,  0.0071,  ..., -0.0181,  0.0241,  0.0366]],
       device='cuda:0', grad_fn=